In [1]:
import os

import json

from tqdm.auto import tqdm

import time

In [2]:
from minio import Minio
import requests
import msgpack

In [3]:
def connect_to_minio(minio_addr, access_key, secret_key):
    # Initialize the MinIO client
    client = Minio(minio_addr, access_key, secret_key, secure=False)

    #Check server status
    try:
        response = requests.get("http://" + minio_addr + "/minio/health/live", timeout=5)
        if response.status_code == 200:
            print("Connected to MinIO server.")
        else:
            return None
    except requests.RequestException as e:
        return None
    
    return client

In [4]:
MINIO_ADDRESS = "123.176.98.90:9000"
access_key = "GXvqLWtthELCaROPITOG"
secret_key = "DmlKgey5u0DnMHP30Vg7rkLT0NNbNIGaM8IwPckD"

In [5]:
client = connect_to_minio(MINIO_ADDRESS, access_key, secret_key)

Connected to MinIO server.


In [6]:
bucket_name = 'datasets'

In [7]:
ROOT = '../kcg-ml-image-pipeline/output/dataset/'
dataset_name = 'environmental'

In [13]:
objects = client.list_objects(bucket_name, dataset_name, recursive=True)

In [ ]:
objects = list(tqdm(objects))

0it [00:00, ?it/s]

In [15]:
for obj in tqdm(objects):
    
    object_name = obj.object_name
    
    # if not object_name.lower().endswith((".jpg", ".png", ".jpeg")):
    #     continue
    # out_path = os.path.join(ROOT, 'image', object_name)
    
    if not object_name.lower().endswith('_clip.msgpack'):
        continue
    out_path = os.path.join(ROOT, 'clip', object_name)
    
    if os.path.exists(out_path):
        continue
        
    try:
        client.fget_object(bucket_name, object_name, out_path)
    except KeyboardInterrupt:
        break
    except:
        continue

  0%|          | 0/1354758 [00:00<?, ?it/s]

In [20]:
image_paths = list()

# for dname in os.listdir(os.path.join(ROOT, 'image', dataset_name)):
#     for fname in os.listdir(os.path.join(ROOT, 'image', dataset_name, dname)):
#         if not fname.endswith('.jpg'):
#             continue
#         image_paths.append(os.path.join(dataset_name, dname, fname))

for dname in os.listdir(os.path.join(ROOT, 'clip', dataset_name)):
    for fname in os.listdir(os.path.join(ROOT, 'clip', dataset_name, dname)):
        if not fname.endswith('_clip.msgpack'):
            continue
        image_paths.append(os.path.join(dataset_name, dname, fname))

In [21]:
for path in tqdm(image_paths):
    
    object_name = path.replace('_clip.msgpack', '_data.msgpack')
    # object_name = path.replace('.jpg', '_data.msgpack')
    out_path = os.path.join(ROOT, 'data', object_name)
    
#     object_name = path.replace('.jpg', '_clip.msgpack')
#     out_path = os.path.join(ROOT, 'clip', object_name)
    
    if os.path.exists(out_path):
        continue
    
    try:
        client.fget_object(bucket_name, object_name, out_path)
    except KeyboardInterrupt:
        break
    except:
        continue
    

  0%|          | 0/75898 [00:00<?, ?it/s]

In [20]:
import msgpack
import numpy as np

In [21]:
with open(out_path, 'rb') as f:
    data = f.read()
decoded_data = msgpack.unpackb(data)

In [22]:
msp_paths = list()
vectors = list()

for dname in os.listdir(os.path.join(ROOT, 'clip', dataset_name)):
    for fname in os.listdir(os.path.join(ROOT, 'clip', dataset_name, dname)):
        
        msp_path = os.path.join(dataset_name, dname, fname)
        
        try:
            with open(os.path.join(ROOT, 'clip', msp_path), 'rb') as f:
                data = f.read()
            decoded_data = msgpack.unpackb(data)
        except:
            client.fget_object(bucket_name, msp_path, os.path.join(ROOT, 'clip', msp_path))
            continue
        
        msp_paths.append(msp_path)
        vectors.append(np.array(decoded_data['clip-feature-vector']))

In [23]:
EMB_PATH = 'data/environmental/clip_vision_emb.npz'

In [24]:
np.savez(
    EMB_PATH,
    file_paths=np.array(msp_paths),
    image_embeds=np.concatenate(vectors, axis=0)
)

# from rank

In [13]:
RANK_DIR = '../kcg-ml-image-pipeline/output/environmental/ranking_v1/ranking_data/'

In [15]:
file_paths = set()

for fname in tqdm(os.listdir(RANK_DIR), leave=False):
    
    info = json.load(open(os.path.join(RANK_DIR, fname)))
    
    image_path_1 = info['image_1_metadata']['file_path']
    
    file_paths.add(image_path_1)
    
    image_path_2 = info['image_2_metadata']['file_path']
        
    file_paths.add(image_path_2)

  0%|          | 0/49097 [00:00<?, ?it/s]

In [19]:
for file_name in tqdm(file_paths):
        
    output_path = file_name.replace('.jpg', '_clip.msgpack').replace(f'datasets/{dataset_name}/', os.path.join(ROOT, f'clip/{dataset_name}/'))
    file_name = file_name.replace('.jpg', '_clip.msgpack').replace('datasets/', '')
        
#     output_path = file_name.replace('.jpg', '_data.msgpack').replace(f'datasets/{dataset_name}/', os.path.join(ROOT, f'data/{dataset_name}/'))
#     file_name = file_name.replace('.jpg', '_data.msgpack').replace('datasets/', '')
        
#     output_path = file_name.replace(f'datasets/{dataset_name}/', os.path.join(ROOT, f'image/{dataset_name}/'))
#     file_name = file_name.replace('datasets/', '')
    
    if os.path.exists(output_path):
        continue
    
    try:
        client.fget_object(bucket_name, file_name, output_path)
    except KeyboardInterrupt:
        break
    except:
        continue
    

  0%|          | 0/30946 [00:00<?, ?it/s]